In [446]:
import pandas as pd
from random import randint
import time
import re
# Scraping through Chrome webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait

In [2]:
# Starting URLs
centris = "https://www.centris.ca/en/properties~for-sale?view=Thumbnail"
duproprio = "https://duproprio.com/en/search/list?search=true&is_for_sale=1&with_builders=1&parent=1&pageNumber=1&sort=-published_at"

In [477]:
class Centris:
    """
    Represents a data object scraped from
    centris.ca. Uses selenium and the Chrome webdriver
    to access# Path to Chromedriver web elements.
    
    Attr:
    self.url - starting url for scraping process
    self.data - pandas.DataFrame object containing scraped data
    self.driver - Chrome webdriver
    self.containers - List of web-elements containing information on listings
        - eg: link to listing, price, picture, address,...
    self.links_to_listings - List of web-elements, each containing the link to a listing
    """
    # Path to Chromedriver
    DRIVER_PATH = 'C:/webdriver/chromedriver.exe'
        
    def __init__(self, url="https://www.centris.ca/en/properties~for-sale?view=Thumbnail"): 
        self.url = url
        self.data = pd.DataFrame(columns=['title', 'price(CAD)', 'address'])
        self.driver = None
        self.containers = None
        self.links_to_listings = None
        
    def add_data(self, title, price, address):
        new_data = pd.DataFrame({'title': title,
                                'price(CAD)': price,
                                 'address': address})
        self.data = self.data.append(new_data)
             
    def get_data(self):
        return self.data
        
    def start_driver(self):
        """
        Starts and returns Crome webdriver. The specified 'self.url' page 
        is opened in headless mode.
        """
        
        # Activate headless mode for fastest response
        options = Options()
        options.add_argument("--headless")

        # Start driver with url
        self.driver = webdriver.Chrome(executable_path=DRIVER_PATH)
        self.driver.get(self.url)
    
    def set_containers(self):
        try:
            containers = self.driver.find_elements_by_class_name("description")
            # Last element is empty string
            self.containers = containers[:-1]
        except:
            print("Chrome driver is not running!")
            print("Use: \"start_driver()\"")
            
    def goto_first_page(self):
        try:
            next_page = self.driver.find_element_by_xpath("//li[@class='goFirst']")
            next_page.click()
        except:
            print("goFirst button not available")
    
    def next_page(self):
        try:
            next_page = self.driver.find_element_by_xpath("//li[@class='next']")
            next_page.click()
        except:
            print("Next page button not available")
            
    def set_links_to_listings(self):
        """Retrieves list of elements from each container with link to listing site"""
        try:
            self.links_to_listings = [container.find_element_by_class_name("a-more-detail") for container in self.containers]
        except:
            if self.driver == None:
                print("Start driver first: start_driver()")
            elif self.containers == None:
                print("Set container attribute: set_containers()" )
            else: print("No class 'a-more-detail' found on current page")
                
    def get_data_from_link(self, link_index:int):
        """
        Retrieves information from link at link_index in  
        links_to_listings list and appends to self.data
        """
        # Web element with link to listing
        element = self.links_to_listings[link_index]
        # Open link in new tab
        actions = ActionChains(self.driver)\
            .key_down(Keys.CONTROL)\
            .click(element)\
            .key_up(Keys.CONTROL)\
            .perform()
        # Move to new tab
        self.driver.switch_to.window(self.driver.window_handles[-1])
        
        # Scrape data
        print("Starting scrape process.")
        # Scroll down 3x to load entire page
        for i in range(3):
            self.driver.find_element_by_tag_name("body")\
                .send_keys(Keys.PAGE_DOWN)
        
        # Get elements on page
        list_container = self.driver.find_element_by_xpath(\
                             "//div[@class='ll-list ps ps--active-y']")
        description = self.driver.find_element_by_xpath(\
                            "//div[@class='col-lg-12 description']")
        print(description.text)
        print(list_container.text)
        # Quit new tab
        self.driver.close()
                                                  
# Instantiate class object
centris = Centris()

In [478]:
# Test
centris.start_driver()
centris.set_containers()
centris.set_links_to_listings()

In [479]:
centris.get_data_from_link(2)

Starting scrape process.
Features
?
Lifestyle
Lot area
64,046 sqft
Zoning
Residential
20
Car friendly
9
Quiet
6
Elementary Schools
6
Greenery
5
Parks
5
Transit friendly
4
High Schools
4
Pedestrian friendly
4


In [470]:
browser = webdriver.Chrome('C:/webdriver/chromedriver.exe')
browser.get("https://www.centris.ca/en/lots~for-sale~terrebonne-lachenaie/12479005?view=Summary&uc=1")
browser.find_element_by_tag_name("body").send_keys(Keys.PAGE_DOWN)

In [473]:
browser.find_element_by_xpath(\
        "//div[@class='ll-list ps ps--active-y']")

<selenium.webdriver.remote.webelement.WebElement (session="7d04f02d53d5f761449e6ce684bac0e3", element="a425e126-6a30-4109-89a3-560e2a4a5b79")>

In [472]:
browser.find_element_by_tag_name("body").send_keys(Keys.PAGE_DOWN)